# Schedule recordings using saved orbital data

In [1]:
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.2.1     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   1.0.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
library(lubridate)


Attaching package: ‘lubridate’

The following object is masked from ‘package:base’:

    date



In [3]:
library(yaml)

In [4]:
library(assertthat)


Attaching package: ‘assertthat’

The following object is masked from ‘package:tibble’:

    has_name



In [5]:
paste("Running at ", Sys.time() %>% .POSIXct("GMT"), "GMT")

[1] "Running at  2019-09-22 18:38:57 GMT"

In [7]:
assert_that(file.exists("satellite_data_df.rds"),
            msg = "No satellite data files found on disk. Have you run `Satellite_orbital_data`?")

[1] TRUE

In [8]:
satellite_data_df <- readRDS("satellite_data_df.rds")

In [9]:
assert_that(file.exists("calibration.yaml"),
            msg = "No dongle calibration data on disk. Have you run `Calibrate_receiver`?")

[1] TRUE

In [14]:
calibration_data <- read_yaml("calibration.yaml")

In [15]:
clock_offset <- calibration_data$clock_offset

## Convert to satellite passes each day

Get the current time in UTC

In [16]:
current_time_utc <- Sys.time() %>% .POSIXct("GMT")

Make sure we have a future satellite pass in the data, then filter out any past passes.

In [17]:
assert_that(nrow(satellite_data_df %>% filter(startUTC > current_time_utc)) > 0,
            msg = "No future satellite orbital data available")

[1] TRUE

In [18]:
satellite_data_df <- satellite_data_df %>% filter(startUTC > current_time_utc)

## Format systemd / mlrpt commands

For systemd, using `systemd-run` rather than the usual template based form since these are one-off non-recurring commands. The argument `--user` runs the command as the user who scheduled it. `--unit` gives the job a name. The argument `--on-calendar` schedules the start and needs to be one minute before the satellite pass to give systemd and mlrpt time to start up. The format is yyyy-mm-dd HH:MM in **local time**. Documentation at https://www.freedesktop.org/software/systemd/man/systemd-run.html

For `mlrpt` need arguments in the form of `mlrpt -s $startTime-$stopTime -t $duration`. Start and stop time are in the format HHMM in **UTC**. Duration is how long the command needs to run (in **minutes**) from start to shut-down. Make it 3 minutes longer than the time of the satellite pass to give time for start-up, and at the end, decoding and saving images.

In [19]:
paste0('systemd-run --user --unit=\"mlrpt_', satellite_data_df$startDate, "_", satellite_data_df$localStartTime,
       '\" --on-calendar=\"', satellite_data_df$startDate," ", satellite_data_df$localStartTime,
       '\" /usr/local/bin/mlrpt -s ', satellite_data_df$startTime,"-", satellite_data_df$endTime,
       ' -t ', satellite_data_df$duration)

[1] "systemd-run --user --unit=\"mlrpt_2019-09-23_07:49\" --on-calendar=\"2019-09-23 07:49\" /usr/local/bin/mlrpt -s 0550-0606 -t 16"
  [2] "systemd-run --user --unit=\"mlrpt_2019-09-23_20:13\" --on-calendar=\"2019-09-23 20:13\" /usr/local/bin/mlrpt -s 1814-1829 -t 16"
  [3] "systemd-run --user --unit=\"mlrpt_2019-09-24_07:30\" --on-calendar=\"2019-09-24 07:30\" /usr/local/bin/mlrpt -s 0531-0546 -t 15"
  [4] "systemd-run --user --unit=\"mlrpt_2019-09-24_19:54\" --on-calendar=\"2019-09-24 19:54\" /usr/local/bin/mlrpt -s 1755-1809 -t 15"
  [5] "systemd-run --user --unit=\"mlrpt_2019-09-24_21:34\" --on-calendar=\"2019-09-24 21:34\" /usr/local/bin/mlrpt -s 1935-1949 -t 14"
  [6] "systemd-run --user --unit=\"mlrpt_2019-09-25_07:10\" --on-calendar=\"2019-09-25 07:10\" /usr/local/bin/mlrpt -s 0511-0526 -t 15"
  [7] "systemd-run --user --unit=\"mlrpt_2019-09-25_08:50\" --on-calendar=\"2019-09-25 08:50\" /usr/local/bin/mlrpt -s 0651-0705 -t 15"
  [8] "systemd-run --user --unit=\"mlrpt_2019-09-25_19:34\" --on-calendar=\"2019-09-25 19:34\" /usr/local/bin/mlrpt -s 1735-1749 -t 14"
  [9] "systemd-run --user --unit=\"mlrpt_2019-09-25_21:14\" --on-calendar=\"2019-09-25 21:14\" /usr/local/bin/mlrpt -s 1915-1929 -t 15"
 [10] "systemd-run --user --unit=\"mlrpt_2019-09-26_08:30\" --on-calendar=\"2019-09-26 08:30\" /usr/local/bin/mlrpt -s 0631-0646 -t 15"
 [11] "systemd-run --user --unit=\"mlrpt_2019-09-26_20:54\" --on-calendar=\"2019-09-26 20:54\" /usr/local/bin/mlrpt -s 1855-1910 -t 16"
 [12] "systemd-run --user --unit=\"mlrpt_2019-09-27_08:10\" --on-calendar=\"2019-09-27 08:10\" /usr/local/bin/mlrpt -s 0611-0626 -t 16"
 [13] "systemd-run --user --unit=\"mlrpt_2019-09-27_20:34\" --on-calendar=\"2019-09-27 20:34\" /usr/local/bin/mlrpt -s 1835-1850 -t 16"
 [14] "systemd-run --user --unit=\"mlrpt_2019-09-28_07:50\" --on-calendar=\"2019-09-28 07:50\" /usr/local/bin/mlrpt -s 0551-0606 -t 16"
 [15] "systemd-run --user --unit=\"mlrpt_2019-09-28_20:14\" --on-calendar=\"2019-09-28 20:14\" /usr/local/bin/mlrpt -s 1815-1830 -t 16"
 [16] "systemd-run --user --unit=\"mlrpt_2019-09-29_07:30\" --on-calendar=\"2019-09-29 07:30\" /usr/local/bin/mlrpt -s 0531-0546 -t 15"
 [17] "systemd-run --user --unit=\"mlrpt_2019-09-29_19:54\" --on-calendar=\"2019-09-29 19:54\" /usr/local/bin/mlrpt -s 1755-1810 -t 15"
 [18] "systemd-run --user --unit=\"mlrpt_2019-09-29_21:35\" --on-calendar=\"2019-09-29 21:35\" /usr/local/bin/mlrpt -s 1936-1949 -t 14"
 [19] "systemd-run --user --unit=\"mlrpt_2019-09-30_07:11\" --on-calendar=\"2019-09-30 07:11\" /usr/local/bin/mlrpt -s 0512-0526 -t 15"
 [20] "systemd-run --user --unit=\"mlrpt_2019-09-30_08:51\" --on-calendar=\"2019-09-30 08:51\" /usr/local/bin/mlrpt -s 0652-0706 -t 15"
 [21] "systemd-run --user --unit=\"mlrpt_2019-09-30_19:35\" --on-calendar=\"2019-09-30 19:35\" /usr/local/bin/mlrpt -s 1736-1750 -t 14"
 [22] "systemd-run --user --unit=\"mlrpt_2019-09-30_21:15\" --on-calendar=\"2019-09-30 21:15\" /usr/local/bin/mlrpt -s 1916-1930 -t 15"
 [23] "systemd-run --user --unit=\"mlrpt_2019-10-01_08:31\" --on-calendar=\"2019-10-01 08:31\" /usr/local/bin/mlrpt -s 0632-0647 -t 15"
 [24] "systemd-run --user --unit=\"mlrpt_2019-10-01_20:55\" --on-calendar=\"2019-10-01 20:55\" /usr/local/bin/mlrpt -s 1856-1911 -t 16"
 [25] "systemd-run --user --unit=\"mlrpt_2019-10-02_08:11\" --on-calendar=\"2019-10-02 08:11\" /usr/local/bin/mlrpt -s 0612-0627 -t 16"
 [26] "systemd-run --user --unit=\"mlrpt_2019-10-02_20:35\" --on-calendar=\"2019-10-02 20:35\" /usr/local/bin/mlrpt -s 1836-1843 -t 8" 
 [27] "systemd-run --user --unit=\"mlrpt_2019-09-22_01:32\" --on-calendar=\"2019-09-22 01:32\" /usr/local/bin/mlrpt -s 2333-2347 -t 14"
 [28] "systemd-run --user --unit=\"mlrpt_2019-09-23_03:12\" --on-calendar=\"2019-09-23 03:12\" /usr/local/bin/mlrpt -s 0113-0127 -t 15"
 [29] "systemd-run --user --unit=\"mlrpt_2019-09-23_15:34\" --on-calendar=\"2019-09-23 15:34\" /usr/local/bin/mlrpt -s 1335-1350 -t 15"
 [30] "systemd-run --user --unit=\"mlrpt_2019-09-24_02:49\

## Format systemd / rtl_fm commands 

http://kmkeen.com/rtl-demod-guide/

`timeout $duration rtl_fm $biast -f $freq -s $sample -g $dongleGain -F 9 -A fast -E offset -p $dongleShift $recdir/$fileNameCore.raw | tee -a $logFile`

-For `timeout`, append `m` to the time for minutes. Seconds is the default

-Whether to turn bias-t power on (i.e., for LNA). See: `rtl_fm --help | grep "-T"`
biast="" 

-Sample rate, width of recorded signal - should include few kHz for doppler shift.
sample='48000'

-Dongle gain. Run `rtl_test` to get supported gain levels. 49.6 seems to be the highest supported by rtl-sdr.
dongleGain='49.6'

-`-F` fir_size (default: off) enables low-leakage downsample filterEnable a higher quality downsampling FIR than the default boxcar filter. `-F 0` is okay to use while `-F 9` is still a work in progress.
FIR=9

-`-A` Choose how arctan is computed. Options select between the standard (floating) lib *std*, a fast polynomial integer approximation *fast* and a precomputed look-up-table *lut*
arctan=fast

dongleshift

- `-E`
    1) `-E dc` dc blocking filter
    2) `-E deemp` de-emphasis filter for WBFM
    3) `-E direct` direct sampling mode
    4) `-E offset` offset tuning mode (E4000 only)
    Use multiple -E option to enable multiple features.


-Sample rate of the wav file. Shouldn't be changed
wavrate='11025'

-Dongle index, is there any rtl_fm allowing passing serial of dongle?
dongleIndex='0'

-enchancements to apply to the pocessed images. See wxtoimg manual for available options
enchancements=('MCIR-precip' 'HVC' 'MSA' 'therm' 'HVCT-precip' 'NO')

-resize images to the given size to avoid growing of the repository; in px, otherwise, comment out the line
resizeimageto=1024

In [24]:
paste0('systemd-run --user --unit=\"rtl_fm_', satellite_data_df$startDate, "_", satellite_data_df$localStartTime,
       '\" --on-calendar=\"', satellite_data_df$startDate," ", satellite_data_df$localStartTime,
       '\" /usr/bin/timeout ', satellite_data_df$duration, 'm', ' /usr/local/bin/rtl_fm -f ', satellite_data_df$frequency,
       ' -s 48000 -g 49.6 -F 9 -A fast -E offset -p ', clock_offset, 
       ' ', satellite_data_df$startDate, "_", satellite_data_df$localStartTime, '.raw')

[1] "systemd-run --user --unit=\"rtl_fm_2019-09-23_07:49\" --on-calendar=\"2019-09-23 07:49\" /usr/bin/timeout 16m /usr/local/bin/rtl_fm -f 137100000 -s 48000 -g 49.6 -F 9 -A fast -E offset -p -0.605 2019-09-23_07:49.raw"
  [2] "systemd-run --user --unit=\"rtl_fm_2019-09-23_20:13\" --on-calendar=\"2019-09-23 20:13\" /usr/bin/timeout 16m /usr/local/bin/rtl_fm -f 137100000 -s 48000 -g 49.6 -F 9 -A fast -E offset -p -0.605 2019-09-23_20:13.raw"
  [3] "systemd-run --user --unit=\"rtl_fm_2019-09-24_07:30\" --on-calendar=\"2019-09-24 07:30\" /usr/bin/timeout 15m /usr/local/bin/rtl_fm -f 137100000 -s 48000 -g 49.6 -F 9 -A fast -E offset -p -0.605 2019-09-24_07:30.raw"
  [4] "systemd-run --user --unit=\"rtl_fm_2019-09-24_19:54\" --on-calendar=\"2019-09-24 19:54\" /usr/bin/timeout 15m /usr/local/bin/rtl_fm -f 137100000 -s 48000 -g 49.6 -F 9 -A fast -E offset -p -0.605 2019-09-24_19:54.raw"
  [5] "systemd-run --user --unit=\"rtl_fm_2019-09-24_21:34\" --on-calendar=\"2019-09-24 21:34\" /usr/bin/timeout 14m /usr/local/bin/rtl_fm -f 137100000 -s 48000 -g 49.6 -F 9 -A fast -E offset -p -0.605 2019-09-24_21:34.raw"
  [6] "systemd-run --user --unit=\"rtl_fm_2019-09-25_07:10\" --on-calendar=\"2019-09-25 07:10\" /usr/bin/timeout 15m /usr/local/bin/rtl_fm -f 137100000 -s 48000 -g 49.6 -F 9 -A fast -E offset -p -0.605 2019-09-25_07:10.raw"
  [7] "systemd-run --user --unit=\"rtl_fm_2019-09-25_08:50\" --on-calendar=\"2019-09-25 08:50\" /usr/bin/timeout 15m /usr/local/bin/rtl_fm -f 137100000 -s 48000 -g 49.6 -F 9 -A fast -E offset -p -0.605 2019-09-25_08:50.raw"
  [8] "systemd-run --user --unit=\"rtl_fm_2019-09-25_19:34\" --on-calendar=\"2019-09-25 19:34\" /usr/bin/timeout 14m /usr/local/bin/rtl_fm -f 137100000 -s 48000 -g 49.6 -F 9 -A fast -E offset -p -0.605 2019-09-25_19:34.raw"
  [9] "systemd-run --user --unit=\"rtl_fm_2019-09-25_21:14\" --on-calendar=\"2019-09-25 21:14\" /usr/bin/timeout 15m /usr/local/bin/rtl_fm -f 137100000 -s 48000 -g 49.6 -F 9 -A fast -E offset -p -0.605 2019-09-25_21:14.raw"
 [10] "systemd-run --user --unit=\"rtl_fm_2019-09-26_08:30\" --on-calendar=\"2019-09-26 08:30\" /usr/bin/timeout 15m /usr/local/bin/rtl_fm -f 137100000 -s 48000 -g 49.6 -F 9 -A fast -E offset -p -0.605 2019-09-26_08:30.raw"
 [11] "systemd-run --user --unit=\"rtl_fm_2019-09-26_20:54\" --on-calendar=\"2019-09-26 20:54\" /usr/bin/timeout 16m /usr/local/bin/rtl_fm -f 137100000 -s 48000 -g 49.6 -F 9 -A fast -E offset -p -0.605 2019-09-26_20:54.raw"
 [12] "systemd-run --user --unit=\"rtl_fm_2019-09-27_08:10\" --on-calendar=\"2019-09-27 08:10\" /usr/bin/timeout 16m /usr/local/bin/rtl_fm -f 137100000 -s 48000 -g 49.6 -F 9 -A fast -E offset -p -0.605 2019-09-27_08:10.raw"
 [13] "systemd-run --user --unit=\"rtl_fm_2019-09-27_20:34\" --on-calendar=\"2019-09-27 20:34\" /usr/bin/timeout 16m /usr/local/bin/rtl_fm -f 137100000 -s 48000 -g 49.6 -F 9 -A fast -E offset -p -0.605 2019-09-27_20:34.raw"
 [14] "systemd-run --user --unit=\"rtl_fm_2019-09-28_07:50\" --on-calendar=\"2019-09-28 07:50\" /usr/bin/timeout 16m /usr/local/bin/rtl_fm -f 137100000 -s 48000 -g 49.6 -F 9 -A fast -E offset -p -0.605 2019-09-28_07:50.raw"
 [15] "systemd-run --user --unit=\"rtl_fm_2019-09-28_20:14\" --on-calendar=\"2019-09-28 20:14\" /usr/bin/timeout 16m /usr/local/bin/rtl_fm -f 137100000 -s 48000 -g 49.6 -F 9 -A fast -E offset -p -0.605 2019-09-28_20:14.raw"
 [16] "systemd-run --user --unit=\"rtl_fm_2019-09-29_07:30\" --on-calendar=\"2019-09-29 07:30\" /usr/bin/timeout 15m /usr/local/bin/rtl_fm -f 137100000 -s 48000 -g 49.6 -F 9 -A fast -E offset -p -0.605 2019-09-29_07:30.raw"
 [17] "systemd-run --user --unit=\"rtl_fm_2019-09-29_19:54\" --on-calendar=\"2019-09-29 19:54\" /usr/bin/timeout 15m /usr/local/bin/rtl_fm -f 137100000 -s 48000 -g 49.6 -F 9 -A fast -E offset -p -0.605 2019-09-29_19:54.raw"
 [18] "systemd-run --user --unit=\"rtl_fm_2019-09-29_21:35\" --on-calendar=\"2019-09-29 21:35\" /usr/bin/timeout 14m /usr/local/bin/rtl_fm -f 137100000 -s 48000 -g 49.6 -F 9 -A fast -E offset -p